In [50]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [62]:
openai = OpenAI()
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("API key not found")
else:
    print("API Key found")

MODEL = 'gpt-5-nano'
openai = OpenAI()

API Key found


In [113]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

In [119]:
link_system_prompt = """You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

    {
        "links": [
            {"type": "about page", "url": "https://full.url/goes/here/about"},
            {"type": "careers page", "url": "https://another.full.url/careers"}

        ]
    }
    """

In [115]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [116]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [99]:
# print(get_links_user_prompt("https://www.edwarddonner.com"))

In [120]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links


In [121]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [122]:
# def fetch_page_and_all_relevant_links(url):
#     content = fetch_website_contents(url)
#     relevant_links = select_relevant_links(url)
#     result = f"##Landing page {content} \n\n ##Relevant links: \n"
#     for link in relevant_links['links']:
#         result += f"\n\n ##Link: {link['type']}\n"
#         result += fetch_website_contents(link['url'])
#     return result

def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result  

In [123]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
moonshotai/Kimi-K2-Thinking
Updated
1 day ago
•
30.3k
•
747
maya-research/maya1
Updated
about 4 hours ago
•
5.26k
•
326
dx8152/Qwen-Edit-2509-Multiple-angles
Updated
3 days ago
•
19.1k
•
392
MiniMaxAI/MiniMax-M2
Updated
2 days ago
•
863k
•
1.22k
deepseek-ai/DeepSeek-OCR
Updated
5 days ago
•
3M
•
2.56k
Browse 1M+ models
Spaces
Running
on
CPU Upgrade
1.78k
1.78k
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
Explore loss curves for training LLMs
Running
on
Zero
276
276
Qwen-Image-2509-MultipleAngles
👀
Qwen-Image

In [124]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [125]:
# def brochure_user_prompt(company_name, url):
#     user_prompt = f"""
# You are looking at a company called: {company_name}
# Here are the contents of its landing page and other relevant pages;
# use this information to build a short brochure of the company in markdown without code blocks.\n\n
# """
#     user_prompt += fetch_page_and_all_relevant_links(url)
#     user_prompt = user_prompt[:5_000]
#     return user_prompt


def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [126]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Thinking\nUpdated\n1 day ago\n•\n30.3k\n•\n748\nmaya-research/maya1\nUpdated\nabout 4 hours ago\n•\n5.26k\n•\n326\ndx8152/Qwen-Edit-2509-Multiple-angles\nUpdated\n3 days ago\n•\n19.1k\n•\n392\nMiniMaxAI/MiniMax-M2\nUpdated\n2 days ago\n•\n863k\n•\n1.22k\ndeepseek-ai/DeepSeek-OCR\nUpdated\n5 days ago\n•\n3M\n•\n2.56k\nBrowse 1M+ models\nSpaces\nRunning\non\nCPU Upgrade\n1.

In [132]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(model='gpt-4.1-mini', messages = [
                                        {'role': 'system', 'content': brochure_system_prompt},
                                        {'role': 'user', 'content': get_brochure_user_prompt(company_name, url)}
                                        ])
    result = response.choices[0].message.content
    display(Markdown(result))

# def create_brochure(company_name, url):
#     response = openai.chat.completions.create(
#         model="gpt-4.1-mini",
#         messages=[
#             {"role": "system", "content": brochure_system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#         ],
#     )
#     result = response.choices[0].message.content
#     display(Markdown(result))

In [133]:
create_brochure("Carbon3D", "https://carbon3d.com/")

Selecting relevant links for https://carbon3d.com/ by calling gpt-5-nano
Found 33 relevant links


# Carbon3D Brochure

---

## About Carbon3D

Carbon3D, branded as **Carbon**, is a leading innovator in 3D printing and additive manufacturing technologies. Their platform integrates **high-speed digital light synthesis (DLS)** with advanced materials, proprietary software, and hardware to revolutionize manufacturing across multiple industries. Carbon’s cutting-edge solutions enable fast, scalable production with superior mechanical properties, driving the next generation of digital manufacturing.

---

## Our Platform

Carbon’s additive manufacturing platform is composed of:

- **Materials** — A broad portfolio including rigid, elastomeric, and dental materials tailored for diverse industrial needs.
- **Software** — Powerful tools like the *Design Engine* and custom production software streamline workflow from design to print.
- **Hardware** — Advanced 3D printers (M3, M2, L1) along with automation accessories such as the Smart Part Washer and Automatic Operation Suite to ensure efficiency and quality at scale.

---

## Industries We Serve

Carbon’s technology empowers production across many sectors:

- **Automotive & Industrial**: Lightweight, strong parts to improve performance and efficiency.
- **MedTech & Dental**: Precision dental labs, digital dentures, flexible partials, clear aligners, and orthodontic solutions.
- **Consumer Products**: Custom footwear, helmets, saddles, wheelchair cushions, and more.

---

## Partnerships & Network

Carbon provides a global production network, connecting customers with certified production and dental partners to facilitate local and scalable manufacturing. Opportunities also exist for businesses to become production partners and expand their manufacturing capabilities.

---

## Commitment to Education and Support

Through the **Carbon Academy**, the company offers comprehensive educational resources including courses, webinars, white papers, and product stewardship information ensuring customers and partners are empowered to succeed.

---

## Company Culture & Careers

Carbon fosters a culture of innovation, collaboration, and excellence, motivated by a mission to transform manufacturing worldwide. The company promotes continuous learning and leadership in technology advancement. 

**Current Job Openings:** Carbon regularly updates its careers portal with opportunities across engineering, software development, customer support, and product design for dynamic professionals eager to shape the future of 3D printing.

---

## Contact & Learn More

Visit Carbon’s website to:

- Explore product details and material options.
- Access support and training resources.
- Join their vibrant community of innovators and industry leaders.

### Website: [carbon3d.com](https://www.carbon3d.com)

---

*Unlock the future of manufacturing. Print with Carbon.*

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)
    
                            

In [141]:
stream_brochure("Carbon3D", "https://www.carbon3d.com/")

Selecting relevant links for https://www.carbon3d.com/ by calling gpt-5-nano
Found 18 relevant links


# Carbon3D Company Brochure

---

## About Carbon3D

Carbon3D, known simply as Carbon, is a leading innovator in the field of 3D printing and additive manufacturing. The company leverages advanced technologies to provide scalable, high-quality manufacturing solutions that transform product development across numerous industries.

Carbon’s platform integrates proprietary hardware, software, and materials to enable efficient digital manufacturing. Its digital light synthesis (DLS) technology delivers rapid production speeds, excellent material properties, and exceptional design freedom that empower customers to reimagine manufacturing.

---

## Our Platform & Technology

- **3D Printing Hardware:** Carbon offers a range of advanced printers, including the M3, M2, and L1 models, designed to meet diverse production needs with precision and speed. Complementary equipment like the Smart Part Washer and Automatic Operation Suite enhance post-processing efficiency.

- **Materials:** Carbon provides an extensive portfolio of engineered resins, including rigid, elastomeric, and dental materials tailored for durability, flexibility, and biocompatibility. Their materials enable production of functional, end-use parts for demanding applications.

- **Software:** The company’s Design Engine and Printer Operating System streamline the design-to-manufacture workflow, giving customers full control over print preparation, monitoring, and optimization with real-time updates and automated features.

---

## Industries & Applications

Carbon’s manufacturing solutions serve a wide range of industries, including:

- **Automotive:** Lightweight, durable parts and prototypes.
- **Industrial:** Custom tooling, production parts.
- **MedTech / Dental:** Digital dentures, orthodontic aligners, flexible partials.
- **Consumer Products:** Footwear, helmets, saddles, wheelchair cushions.

Their technology enables manufacturers to accelerate innovation cycles, reduce waste, and customize products at scale.

---

## Customers & Partners

Carbon collaborates with leading brands and manufacturers worldwide through its extensive Production and Dental Partner Networks. These partnerships facilitate localized manufacturing with consistent quality and quick turnaround. The company empowers partners to adopt digital manufacturing and expand their product offerings.

---

## Company Culture

At Carbon, innovation, collaboration, and sustainability are at the heart of the company culture. The team values continuous learning and provides resources such as the Carbon Academy, webinars, and educational courses to support professional growth. They are committed to responsible manufacturing practices and product stewardship.

---

## Careers at Carbon

Carbon is always looking for talented individuals passionate about transforming manufacturing and technology. Current job openings span roles in engineering, software development, materials science, customer support, and operations. Employees enjoy a dynamic, inclusive work environment that fosters creativity and impact.

Explore opportunities and take a virtual tour of the company to learn more about working at Carbon.

---

## Connect with Carbon

- Visit the website to explore products and services: [carbon3d.com](https://www.carbon3d.com)
- Discover resources such as white papers, blogs, and customer stories.
- Contact Customer Support and find production or dental partners near you.
- Access Carbon’s online shop for materials and product solutions.

---

**Carbon3D** is your partner in digital manufacturing innovation, advancing 3D printing to build the future of production. Whether you are a manufacturer, designer, or healthcare provider, Carbon delivers transformative tools and materials to accelerate your success.